In [167]:
abstract ERC20 = {
    flags startcat = Description;
    cat
        Description;
        Expression;
        Variable;
        BoolOperator;
        ArithmeticOperator;
    fun
        mkDescription: Expression -> Description;
        mkComparativeExpression: Variable -> BoolOperator -> Variable -> Expression;
        mkSuccessTransferExpression: Expression -> Expression -> BoolOperator -> Variable -> Expression;
        mkTransferOldBalance: Variable -> BoolOperator -> Variable -> Expression;
        mkTransferSameAddress: BoolOperator -> Variable -> BoolOperator -> Variable -> Expression;
        plus, minus: ArithmeticOperator;
        equals, isNotEqual, and, or: BoolOperator;
        totalSupply, supply, balancesOwner, balance, allowed, remaining, senderAdress, senderNewBalance, oldBalance, transferredValue, recipientAddress, isSuccessful, isNotSuccessful: Variable;
}

Abstract changed, previous concretes discarded.


In [168]:
concrete ERC20Formal of ERC20 = {
    lincat
        Expression = {str: Str};
        Description = {str: Str};
        Variable = {name: Str; desc: Str};
        BoolOperator = {str: Str};
        ArithmeticOperator = {str: Str};
    lin
        equals = {str = "=="};
        isNotEqual = {str = "!="};
        and = {str = "&&"};
        or = {str = "||"};
        minus = {str = "-"};
        totalSupply = {name = "_totalSupply"; desc = "_totalSupply"};
        supply = {name = "supply"; desc = "supply"};
        balancesOwner = {name = "_balances[owner]"; desc = "_balances[owner]"};
        balance = {name = "balance"; desc = "balance"};
        allowed = {name = "_allowed[owner][spender]"; desc = "_allowed[owner][spender]"};
        remaining = {name = "remaining"; desc = "remaining"};
        senderAdress = {name = "msg.sender"; desc = "msg.sender"};
        senderNewBalance = {name = "_balances[msg.sender]"; desc = "_balances[msg.sender]"};
        oldBalance = {name = "__verifier_old_uint(_balances[msg.sender])"; desc = "__verifier_old_uint"};
        transferredValue = {name = "vaAlue"; desc = "value"};
        recipientAddress = {name = "to"; desc = "to"};
        isSuccessful = {name = "success"; desc = "success"};
        isNotSuccessful = {name = "!success"; desc = "!success"};
        mkComparativeExpression var1 boolOp var2 = {str = var1.name ++ boolOp.str ++ var2.name};
        mkTransferKeepBalance var1 boolOp var2 = {str = "(" ++ var1.name ++ boolOp.str ++ var2.name ++ ")"};
        mkTransferSameAddress boolOp1 var1 boolOp2 var2 = {str = boolOp1.str ++ var1.name ++ boolOp2.str ++ var2.name};
        mkSuccessTransferExpression exp1 exp2 boolOp var = {str = exp1.str ++ exp2.str ++ boolOp.str ++ var.name};
        mkDescription exp = exp;
}

Defined ERC20Formal

In [169]:
concrete ERC20Eng of ERC20 = {
    lincat
        Expression = {str: Str};
        Description = {str: Str};
        Variable = {name: Str; desc: Str};
        BoolOperator = {str: Str};
        ArithmeticOperator = {str: Str};
    lin
        equals = {str = variants {"should be equal to" ; "is equal to"}};
        isNotEqual = {str = "is not equal to"};
        and = {str = variants {"and" ; "if"}};
        or = {str = "or"};
        minus = {str = "minus"};
        supply = {name = "supply"; desc = "resulting total supply of tokens"};
        totalSupply = {name = "_totalSupply"; desc = "total supply of tokens"};
        balancesOwner = {name = "_balances[owner]"; desc = "resulting balance of the specified address"};
        balance = {name = "balance"; desc = "balance of the specified address"};
        allowed = {name = "_allowed[owner][spender]"; desc = "resulting allowance of the specified address"};
        remaining = {name = "remaining"; desc = "allowance of the specified address"};
        senderAdress = {name = "msg.sender"; desc = "sender address"};
        senderNewBalance = {name = "_balances[msg.sender]"; desc = "sender new balance"};
        oldBalance = {name = "__verifier_old_uint"; desc = "old balance"};
        transferredValue = {name = "value"; desc = "transferred value"};
        recipientAddress = {name = "to"; desc = "recipient address"};
        isSuccessful = {name = "success"; desc = "is successful"};
        isNotSuccessful = {name = "success"; desc = "is not successful"};
        mkComparativeExpression var1 boolOp var2 = {str = "the" ++ var1.desc ++ boolOp.str ++ "the" ++ var2.desc};
        mkTransferOldBalance var1 boolOp var2 = {str = "the" ++ var1.desc ++ boolOp.str ++ "their" ++ var2.desc};
        mkTransferSameAddress boolOp1 var1 boolOp2 var2 = {str = boolOp1.str ++ "the" ++ var1.desc ++ boolOp2.str ++ "the" ++ var2.desc};
        mkSuccessTransferExpression exp1 exp2 boolOp var = {str = exp1.str ++ exp2.str ++ boolOp.str ++ "the transfer" ++ var.desc};
        mkDescription exp = exp;
}

Defined ERC20Eng

### function totalSupply() external view returns (uint256);

In [144]:
parse -lang=ERC20Eng "the resulting total supply of tokens should be equal to the total supply of tokens" | linearize -lang=ERC20Formal

supply == _totalSupply


### function balanceOf(address who) external view returns (uint256);

In [25]:
parse -lang=ERC20Eng "the resulting balance of the specified address should be equal to the balance of the specified address" | linearize -lang=ERC20Formal

_balances[owner] == balance


### function allowance(address owner, address spender) external view returns (uint256);

In [26]:
parse -lang=ERC20Eng "the resulting allowance of the specified address should be equal to the allowance of the specified address" | linearize -lang=ERC20Formal

_allowed[owner][spender] == remaining


### function transfer(address to, uint256 value) external returns (bool);

In [131]:
parse -lang=ERC20Eng "the sender new balance should be equal to their old balance"

mkDescription (mkTransferKeepBalance senderNewBalance equals oldBalance)


In [132]:
parse -lang=ERC20Eng "if the sender address is equal to the recipient address"

mkDescription (mkTransferSameAddress and senderAdress equals recipientAddress)


In [170]:
parse -lang=ERC20Eng "the sender new balance should be equal to their old balance if the sender address is equal to the recipient address and the transfer is successful" | linearize

the sender new balance should be equal to their old balance and the sender address should be equal to the recipient address and the transfer is successful
[mkTransferOldBalance] && msg.sender == to && success


In [ ]:
parse -lang=ERC20Eng 